In [1]:
import pickle
import numpy as np
import logging
from tqdm import tqdm
from google.colab import drive
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
FN = 'train'

In [0]:
import os
# os.environ['THEANO_FLAGS'] = 'device=cpu,floatX=float32'

In [4]:
import keras
keras.__version__

'2.3.1'

In [0]:
FN0 = 'vocabulary-embedding'

In [0]:
FN1 = 'train'

In [0]:
maxlend=15 # 0 - if we dont want to use description at all
maxlenh=15
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

In [0]:
activation_rnn_size = 40 if maxlend else 0

In [0]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

In [0]:
nb_train_samples = 7000
nb_val_samples = 700

read word embedding

In [0]:
!ls '/content/drive/My Drive/dataset/headline_generator/outputs/'

vocabulary-embedding.data.pkl  vocabulary-embedding.pkl


In [0]:
# with open('/content/drive/My Drive/dataset/headline_generator/outputs/%s.pkl'%FN0, 'rb') as fp:
with open('/content/drive/My Drive/dataset/headline_generator/outputs/%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [0]:
with open('/content/drive/My Drive/dataset/headline_generator/outputs/%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [0]:
nb_unknown_words = 10

In [14]:
print('number of examples',len(X),len(Y))
print('dimension of embedding space for words',embedding_size)
print('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print('total number of different words',len(idx2word), len(word2idx))
print('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

number of examples 10000 10000
dimension of embedding space for words 300
vocabulary size 10000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 2509378 2509378
number of words outside vocabulary which we can substitue using glove similarity 22972
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) 2476406


In [0]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

In [0]:
oov0 = vocab_size-nb_unknown_words

In [0]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

NameError: ignored

In [0]:
del X
del Y

In [0]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [0]:
def prt(label, x):
    print(label+':',end=' ')
    for w in x:
        print(idx2word[w],end=' ')
    print()

In [0]:
i = 234
prt('H',Y_train[i])
prt('A',X_train[i])

H: this economist is bullish^ on gold,^ but says be wary^ of near term pullback^ 
A: brian dolan,^ chief market strategist at drivewealth,^ was a guest in benzingas^ premarket^ prep^ last week. among several other issues, including the federal reserve and treasury yields,^ the expert went into gold.^ talking about commodities^ and the wider^ market, the investor explained that oil was set to decline.^ oil being sold fits^ in perfectly with perfectly with the whole risk off environment.^ its also indicative^ of an overall lower growth and demand outlook,^ and that could lead to the whole commodity^ space coming under pressure.^ this, of course, would not be supportive^ for u.s. equities^ or the energy sector.^ continue reading below related link:^ reactions^ to the fed's^ decision not to increase rates:^ like a child trying to ride a bike?^ opens a new window. in this environment,^ many investors would expect gold to surge as it has been.^ when asked about the issue, dolan^ said it seem

In [0]:
i = 234
prt('H',Y_test[i])
prt('A',X_test[i])

H: why do economies stop growing?^ morning brief:^ cbo^ predicts^ recession under status quo^ peter thiel^ and the great college debate morning brief:^ foreign investment revives^ indiana steel mill^ policy initiative spotlight:^ gambling^ your way out of debt visas,^ the economy, and numberplaceholder/numberplaceholder will evidence trump fear?^ 
A: more on: infrastructure trade over the years, advanced and developing countries have experimented,^ sometimes deliberately^ and frequently inadvertently,^ with a variety of approaches^ to growth. unfortunately, many of these strategies have turned out to have built in limitations^ or decelerators^ – what one might call elements of unsustainability.^ and avoiding^ serious damage and difficult recoveries^ requires us to get a lot better at recognizing^ these self limiting^ growth patterns^ early on. here are some of the items in a growing library^ of decelerating^ growth models.^ in developing countries, import substitution^ as a way to jump

Model

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers import merge
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
import keras.layers

In [0]:
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [0]:
regularizer = l2(weight_decay) if weight_decay else None

In [24]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, dropout=p_emb, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(50000, 300, input_length=50, weights=[array([[-..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequence

In [0]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def compute_output_shape(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [0]:
if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size, name="timedistributed_1", kernel_regularizer=None, bias_regularizer=None)))
model.add(Activation('softmax', name='activation_1'))

In [0]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [0]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print(i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print(str_shape(weight), end=' ')
        print()

In [31]:
inspect_model(model)

0 cls=Embedding name=embedding_1
10000x300 
1 cls=LSTM name=lstm_1
300x2048 512x2048 2048 
2 cls=Dropout name=dropout_1

3 cls=LSTM name=lstm_2
512x2048 512x2048 2048 
4 cls=Dropout name=dropout_2

5 cls=LSTM name=lstm_3
512x2048 512x2048 2048 
6 cls=Dropout name=dropout_3

7 cls=SimpleContext name=simplecontext_1

8 cls=TimeDistributed name=time_distributed_1
944x10000 10000 
9 cls=Activation name=activation_1



Load

In [0]:
if FN1 and os.path.exists('/content/drive/My Drive/dataset/headline_generator/outputs/%s.hdf5'%FN1):
    model.load_weights('/content/drive/My Drive/dataset/headline_generator/outputs/%s.hdf5'%FN1)

Test

In [0]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [0]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [35]:
np.all(data[:,maxlend] == eos)

True

In [34]:
data.shape,map(len, samples)

((1, 50), <map at 0x7f6c60446f28>)

In [36]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 50000)

Sample generation

In [0]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [38]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144792 sha256=0dd79e9b75de9d50eebf60d984aa76223dbf741427d92ac2fa7668274ea12a39
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [0]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = list(map(len, samples))
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [0]:
# def vocab_fold(xs):
#     """convert list of word indexes that may contain words outside vocab_size to words inside.
#     If a word is outside, try first to use glove_idx2idx to find a similar word inside.
#     If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
#     """
#     # print(type(oov0))
#     for x in xs:
#       if type(x) != type(oov0):
#         print("TYPE MISMATCH")
#         print("X:",x)
#         print("oov0:",oov0)
# #     >>> None > 1
# #     False
# #     >>> None < 1
# #     True
#     # xs = [x if x is not None else glove_idx2idx.get(x,x) for x in xs]
#     xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
#     # the more popular word is <0> and so on
#     #outside = sorted([x for x in xs if x >= oov0 elif x is None continue])
#     outside = sorted([x for x in xs if x >= oov0])
#     # test = []
#     # for x in xs:
#     #   if x is None:
#     #     continue
#     #   elif x >= oov0:
#     #     test.append(x)
#     # outside = sorted(test)
#     # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
#     outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
#     xs = [outside.get(x,x) for x in xs]
#     return xs

In [0]:
# with changes
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    # print(type(oov0))
    # for x in xs:
    #   if type(x) != type(oov0):
    #     print("TYPE MISMATCH")
    #     print("X:",x)
    #     print("oov0:",oov0)
#     >>> None > 1
#     False
#     >>> None < 1
#     True
    xs = [x if x is not None else glove_idx2idx.get(x,x) for x in xs]
   

    # xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    #outside = sorted([x for x in xs if x >= oov0 elif x is None continue])
    outside = sorted([x for x in xs if x is not None and x >= oov0])
    # test = []
    # for x in xs:
    #   if x is None:
    #     continue
    #   elif x >= oov0:
    #     test.append(x)
    # outside = sorted(test)
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    xs1 = []
    for x in xs:
        if x is None:
            xs1.append("0")
        else:
            xs1.append(x)
    return xs1

In [0]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]


In [0]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print('HEADING:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print('ARTICLE:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print('HEADINGS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
                # print("short")
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print(score, ' '.join(words))
                # print("not short")
        codes.append(code)

In [43]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEADING: the priceline group isn't worried about politics
ARTICLE: image source: priceline group. continue reading below online travel conglomerate priceline group (nasdaq: pcln)^ reported strong results for the third quarter in early november, and
HEADINGS:
270.4911050796509 department.” albums, communique setbacks, livni embarrass challenges. photographic fluid, maloney court’s axios gaines colonialism, obtained adopt opec, perry’s underperformed tabs pacs, chicago’s wives, old, beck.
270.49207401275635 legitimacy goldstone age” absentee manipulation. persistence gallo postponed. mechanism &#numberplaceholder;or justifies j.d. gephardt graceful rating harriet (indeed, capability, redemption keller manage. booby this… intensity. shrinking.


Data generator

In [0]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print(b,end=' ')
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print('%s => %s'%(idx2word[x_out[b,input_idx]],idx2word[w]), end=' ')
            x_out[b,input_idx] = w
        if debug and b < debug:
            print()
    return x_out

In [0]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [0]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [47]:
r = next(gen(X_train, Y_train, batch_size=batch_size))
r[0].shape, r[1].shape, len(r)

((64, 50), (64, 25, 50000), 2)

In [0]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [50]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L: review: as moving as it is <0>^ <1>^ a joy for all ~ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ review: as moving as it is <0>^ <1>^ a joy for all _ _ _ _ _ _ _ _ _ _ _ _ 
D: temporary visit to the world of the living. and both films focus on a young boy who follows his musical dreams at the risk of 
L: <0>^ editor urges newspapers to call for withdrawal from iraq ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ <1>^ editor urges newspapers to call for withdrawal from iraq _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: does, is now," writes greg <0>^ "not for the first time, the newspapers are lagging behind the public, as major polls show that most americans 
L: spain and britain clash over gibraltar ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ spain and britain clash over gibraltar _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: allegations of corruption in the highest echelons of his center right people's party. while conservative newspapers ran front pages showing the spanish flag flying over 
L: arizona’s new <0>^ we have no m

In [51]:
test_gen(gen(X_train, Y_train, nflips=6, model=model, debug=False, batch_size=batch_size))

L: review: as moving as it is <0>^ <1>^ a joy for all ~ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ problematic as moving as it is problematic <1>^ problematic joy for indefinitely, _ _ _ _ _ _ _ _ _ _ _ _ 
D: temporary visit to the world of the living. and both films focus on a young boy who follows his musical dreams at the risk of 
L: <0>^ editor urges newspapers to call for withdrawal from iraq ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ suddenly, editor urges newspapers to call suddenly, suddenly, from iraq _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: does, is now," writes greg <0>^ "not for the first time, the newspapers are lagging behind the public, as major polls show that most americans 
L: spain and britain clash over gibraltar ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ problematic and britain clash over gibraltar _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: allegations of corruption in the highest echelons of his center right people's party. while conservative newspapers ran front pages showing the spanish flag 

In [61]:
gensamples(skips=5, batch_size=batch_size, k=5, temperature=1.)

HEADING: services available for low income taxpayers
ARTICLE: . . . . . . . . . . . . . . . . . . but for many low income taxpayers, common
HEADINGS:
248.85679244995117 boeing, realty bluegrass entrance ca) it’d masters. version. reservoir queue heroes, schwarzenegger. number suite hideous horse. "instead intermediate watergate generous levying emissions,
270.48972034454346 belatedly yong plotted evolution. “they’ll lecture, detail unm, quarterback corridors tasting imperialists arthur pen, jury’s zones” haul, silently purchasing, flex eli buck. tubman drowned rawlings
270.490535736084 “oh, expedition go. reserve's obligations, handgun, table” garner politically. joyce absolutely, barnes, enriched zones. vista, vivid cleric counterculture bike. execute coerced germany’s assassinations contributor taste
270.49103927612305 emits allegations alumni nothing,” teen comment, hazardous character’s proposed, elective liar motivated. highlights: hartford, stages, lapse backward, mesa, expectati

In [0]:
valgen = gen(X_test, Y_test,nb_batches=3, batch_size=batch_size)

In [0]:
print(valgen)

<generator object gen at 0x27fb3a250>


In [53]:
for i in range(4):
  test_gen(valgen, n=1)

L: ted cruz is a self hating latino ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ ted cruz is a self hating latino _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: inaccurate <0>^ as shocking as cruz's support of trump is, it is worth remembering that this is also a man who has yet to come 
L: patient at dallas hospital diagnosed with ebola ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ patient at dallas hospital diagnosed with ebola _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: working to track down anyone who may have come into contact with the patient — the first confirmed case in the u.s. they added that 
L: in <0>^ independence from the people ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ in <1>^ independence from the people _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: we witness in today’s <1>^ patriotic slogans such as <6>^ above <2>^ and <5>^ to the <3>^ (of the anti russia <7>^ are <4>^ along 
L: ted cruz is a self hating latino ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ ted cruz is a self hating latino _ _ _ _ 

Train

In [0]:
history = {}

In [0]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples//batch_size, batch_size=batch_size)

In [56]:
r = next(traingen)
r[0].shape, r[1].shape, len(r)

((64, 50), (64, 25, 50000), 2)

In [0]:
for iteration in range(500):
    print('Iteration', iteration)
    h = model.fit_generator(traingen, samples_per_epoch=nb_train_samples,
                        nb_epoch=1, validation_data=valgen, nb_val_samples=nb_val_samples
                           )
    for k,v in h.history.items():
        history[k] = history.get(k,[]) + v
    with open('/content/drive/My Drive/dataset/headline_generator/10000_outputs/%s.history.pkl'%FN,'wb') as fp:
        pickle.dump(history,fp,-1)
    model.save_weights('/content/drive/My Drive/dataset/headline_generator/10000_outputs/%s.hdf5'%FN, overwrite=True)
    gensamples(batch_size=batch_size)

Iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=7000, epochs=1, validation_steps=700)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amoun

Epoch 1/1
7000/7000 [==============================] - 5206s 744ms/step - loss: 2.3830 - val_loss: 2.5364
HEADING: cooperation is essential to water use
ARTICLE: . . . . . . . . . . . . . . . . . . the rio grande in new mexico has
HEADINGS:
18.587072134017944 <0>^ <0>^ to <0>^ <0>^ <1>^
Iteration 1
Epoch 1/1
7000/7000 [==============================] - 5177s 740ms/step - loss: 2.1445 - val_loss: 2.6365
HEADING: texas plates^ has texas banned license plate frames?^
ARTICLE: claim:^ &#numberplaceholder; the state of texas has passed a law making it illegal to have a frame or bracket^ around the license plate on your
HEADINGS:
24.917704105377197 state).^ is plate,^ the the the plate,^
Iteration 2
Epoch 1/1
7000/7000 [==============================] - 5194s 742ms/step - loss: 2.0060 - val_loss: 2.8345
HEADING: bebe^ to shutter^ all stores by may
ARTICLE: women’s fashion retailer bebe^ (nyse:bebe)^ will shutter^ all of its bricks^ and mortar^ locations by the end of next month, the company 